# Preparação Dos Dados


# **MODELO 1**

In [16]:
import pandas as pd
import numpy as np
import os

file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Função para criar defasagens
def criar_defasagens(data, entradas, coluna):
    for i in range(entradas, 0, -1):  # Defasagens de entradas a 1
        data[f'Entrada {i}'] = data[coluna].shift(i)
    novas_colunas = [f'Entrada {i}' for i in range(entradas, 0, -1)] + [coluna]
    data = data[novas_colunas].copy()
    for i in range(1, 12):  # Defasagens de 1 a saídas
        data[f'Saida {i}'] = data[coluna].shift(-i)
    return data

# Criar datasets para 15, 30 e 45 defasagens
data_15 = criar_defasagens(data.copy(), 15, "Vazao")
data_30 = criar_defasagens(data.copy(), 30, "Vazao")
data_45 = criar_defasagens(data.copy(), 45, "Vazao")




In [17]:
# Pasta onde os arquivos serão salvos

output_folder = 'data/modelo_1'

# Criar a pasta se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos para os arquivos Excel
file_path_15 = os.path.join(output_folder, 'data_15_dias.xlsx')
file_path_30 = os.path.join(output_folder, 'data_30_dias.xlsx')
file_path_45 = os.path.join(output_folder, 'data_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel na pasta específica
data_15.to_excel(file_path_15, index=False)
data_30.to_excel(file_path_30, index=False)
data_45.to_excel(file_path_45, index=False)


# **MODELO 2**



In [20]:
import pandas as pd
import numpy as np
import sys

# Carregar o arquivo Excel
file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Adicionar colunas V1 e V2
data['V1'] = np.nan
data['V2'] = np.nan

# Calcular V1 baseado na coluna Vazao
data.loc[1:, 'V1'] = ((data['Vazao'].iloc[1:] - data['Vazao'].shift(1).iloc[1:]) / data['Vazao'].shift(1).iloc[1:]) * 100

# Calcular V2 baseado na coluna V1
data.loc[1:, 'V2'] = data['V1'].iloc[1:] - data['V1'].shift(1).iloc[1:]

colunas = ['V2', 'V1', 'Vazao']
data = data[colunas]

data


,V2,V1,Vazao
0,NaN,NaN,2110
1,NaN,-0.995261,2089
2,-0.871661,-1.866922,2050
3,-0.084298,-1.951220,2010
4,1.005946,-0.945274,1991
...,...,...,...
17284,-1.364289,0.389105,5418
17285,-3.028455,-2.639350,5275
17286,-1.531266,-4.170616,5055
17287,-0.517811,-4.688427,4818


In [32]:
# Define os intervalos de defasagem a serem testados
intervalos = [15, 30, 45]

dataframes = {
    "V1": {
        15: None,
        30: None,
        45: None
    },
    "V2": {
        15: None,
        30: None,
        45: None
    },
    "Vazao": {
        15: None,
        30: None,
        45: None
    }
}

for coluna in colunas:
    for intervalo in intervalos:
        dataframe = criar_defasagens(data.copy(), intervalo, coluna)
        dataframes[coluna][intervalo] = dataframe

concat15 = pd.concat([dataframes["V2"][15], dataframes["V1"][15], dataframes["Vazao"][15]])
concat30 = pd.concat([dataframes["V2"][30], dataframes["V1"][30], dataframes["Vazao"][30]])
concat45 = pd.concat([dataframes["V2"][45], dataframes["V1"][45], dataframes["Vazao"][45]])

output_folder = 'data/modelo_2'

# Criar a pasta se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos para os arquivos Excel
file_path_15 = os.path.join(output_folder, 'data_15_dias.xlsx')
file_path_30 = os.path.join(output_folder, 'data_30_dias.xlsx')
file_path_45 = os.path.join(output_folder, 'data_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel na pasta específica
concat15.to_excel(file_path_15, index=False)
concat30.to_excel(file_path_30, index=False)
concat45.to_excel(file_path_45, index=False)


{15:        Entrada 15  Entrada 14  Entrada 13  Entrada 12  Entrada 11  Entrada 10  \
 0             NaN         NaN         NaN         NaN         NaN         NaN   
 1             NaN         NaN         NaN         NaN         NaN         NaN   
 2             NaN         NaN         NaN         NaN         NaN         NaN   
 3             NaN         NaN         NaN         NaN         NaN         NaN   
 4             NaN         NaN         NaN         NaN         NaN         NaN   
 ...           ...         ...         ...         ...         ...         ...   
 17284    0.250107   -0.838210   -1.016536    1.609160    0.276229    1.052834   
 17285   -0.838210   -1.016536    1.609160    0.276229    1.052834   -0.000754   
 17286   -1.016536    1.609160    0.276229    1.052834   -0.000754   -0.285847   
 17287    1.609160    0.276229    1.052834   -0.000754   -0.285847   -2.305543   
 17288    0.276229    1.052834   -0.000754   -0.285847   -2.305543   -1.646500   
 
        En